### This script contains:

#### 1. Import libraries & dataset
#### 2. Flags for Station uses
#### 3. Querying low Station uses
#### 4. Removing all Station ID's with less than 100 hires
#### 5. Splitting out Unique Stations

## 1. Import libraries & dataset

In [22]:
import quandl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os


In [23]:
#Ensuring that the graphs you create are displayed within the notebook without the need to "call" them specifically.
%matplotlib inline

In [24]:
#Folder path into usable string
path = r'C:\Users\willm\Dropbox\1 Data Analytics Course\1 New York Citibike Hire'

In [25]:
#Import Hire Dataset
NYB2020_Stations = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'NYB2020merged.pkl'))

In [26]:
#Set Pandas to show all columns
pd.set_option("display.max_columns", None)

## 2. Flags for Station uses

Still using the NYB2020merged.pkl from Script 1, as I would like ALL uses - including the long ones - to be used.

In [27]:
NYB2020_Stations.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,start_hour,start_date2,temp,rain_snow,day_of_week,day_of_week_number,month,month_number,TripMins,age
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,1,0,2020-01-01,3,0.0,Wednesday,3,January,1,13.0,28
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,1,0,2020-01-01,3,0.0,Wednesday,3,January,1,26.0,51
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,1,0,2020-01-01,3,0.0,Wednesday,3,January,1,24.0,57
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,1,0,2020-01-01,3,0.0,Wednesday,3,January,1,10.0,40
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,1,0,2020-01-01,3,0.0,Wednesday,3,January,1,12.0,38


In [28]:
#Creating a new column to allow me to GroupBy the Station IDs columns
NYB2020_Stations['1col'] = "1"

In [29]:
NYB2020_Stations['1col'] = NYB2020_Stations['1col'].astype('int8')

In [30]:
#Creating a GroupBy to give total uses by each 'age' in a new column.
NYB2020_Stations["uses_StartStation"] = NYB2020_Stations.groupby("start_station_id")['1col'].transform("sum")

In [31]:
NYB2020_Stations["uses_EndStation"] = NYB2020_Stations.groupby("end_station_id")['1col'].transform("sum")

In [32]:
NYB2020_Stations[['start_station_id', 'uses_StartStation']].head(20)

,start_station_id,uses_StartStation
0,504,59142.0
1,3423,48873.0
2,3687,73560.0
3,346,38629.0
4,372,8071.0
5,394,28740.0
6,3093,33556.0
7,514,97415.0
8,274,21312.0
9,467,18195.0


Stations 514 shows up 3 times, with the same uses.

In [33]:
NYB2020_Stations[['end_station_id', 'uses_EndStation']].head(20)

,end_station_id,uses_EndStation
0,307,52211.0
1,3300,16094.0
2,259,31685.0
3,490,62776.0
4,3637,16257.0
5,478,62586.0
6,3668,15573.0
7,426,101767.0
8,416,13779.0
9,3368,17005.0


Station 405 shows up twice, with the same uses.

## 3. Querying low Station uses

In [34]:
#Checking to see how many times Stations are used as a starting point - max and min
NYB2020_Stations['start_station_id'].value_counts(dropna = False, ascending=True)

4332         1
4310         2
4328         3
3912         3
4306         6
         ...  
499      91856
514      97415
435      99191
426      99364
3141    100753
Name: start_station_id, Length: 1213, dtype: int64

In [36]:
NYB2020_Stations['start_station_idSTR'] = NYB2020_Stations['start_station_id'].astype('str')

In [38]:
NYB2020_StartStation4332 = NYB2020_Stations[NYB2020_Stations['start_station_idSTR']=='4332']

In [39]:
NYB2020_StartStation4332.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,start_hour,start_date2,temp,rain_snow,day_of_week,day_of_week_number,month,month_number,TripMins,age,1col,uses_StartStation,uses_EndStation,start_station_idSTR
19504238,318,2020-12-31 20:17:25.1670,2020-12-31 20:22:43.9950,4332,Morris Ave & E 163 St,40.82723,-73.917656,4169,Morris Ave & E 171 St,40.83899,-73.9109,48231,Subscriber,1984,2,20,2020-12-31,7,1.0,Thursday,4,December,12,5.0,36,1,1.0,75.0,4332


<font color='blue'>First used on the last day of our data!</font>

In [40]:
NYB2020_StartStation4310 = NYB2020_Stations[NYB2020_Stations['start_station_idSTR']=='4310']

In [41]:
NYB2020_StartStation4310.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,start_hour,start_date2,temp,rain_snow,day_of_week,day_of_week_number,month,month_number,TripMins,age,1col,uses_StartStation,uses_EndStation,start_station_idSTR
19484524,342,2020-12-31 12:01:46.9430,2020-12-31 12:07:29.3590,4310,24 St & 38 Ave,40.756351,-73.936515,3586,34 Ave & 13 St,40.763875,-73.936485,46577,Subscriber,1986,1,12,2020-12-31,7,1.0,Thursday,4,December,12,6.0,34,1,2.0,3354.0,4310
19488691,264,2020-12-31 13:47:57.8940,2020-12-31 13:52:22.6310,4310,24 St & 38 Ave,40.756351,-73.936515,3745,36 Ave & 31 St,40.756513,-73.929575,48559,Subscriber,1985,1,13,2020-12-31,7,1.0,Thursday,4,December,12,4.0,35,1,2.0,10599.0,4310


<font color='blue'>First used on the last day of our data!</font>

In [42]:
NYB2020_StartStation3912 = NYB2020_Stations[NYB2020_Stations['start_station_idSTR']=='3912']

In [43]:
NYB2020_StartStation3912.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,start_hour,start_date2,temp,rain_snow,day_of_week,day_of_week_number,month,month_number,TripMins,age,1col,uses_StartStation,uses_EndStation,start_station_idSTR
161165,278,2020-01-06 10:15:25.1070,2020-01-06 10:20:03.1980,3912,MTL-AOS-5.1,40.7903,-73.9597,3912,MTL-AOS-5.1,40.7903,-73.9597,37965,Subscriber,1996,1,10,2020-01-06,5,2.0,Monday,1,January,1,5.0,24,1,3.0,3.0,3912
214920,486,2020-01-07 14:31:17.8780,2020-01-07 14:39:24.1440,3912,MTL-AOS-5.1,40.7903,-73.9597,3912,MTL-AOS-5.1,40.7903,-73.9597,37965,Subscriber,1996,1,14,2020-01-07,5,0.0,Tuesday,2,January,1,8.0,24,1,3.0,3.0,3912
2551333,2063,2020-03-04 13:24:20.8520,2020-03-04 13:58:43.8620,3912,MTL-AOS-5.1,40.7903,-73.9597,3912,MTL-AOS-5.1,40.7903,-73.9597,37965,Customer,1969,0,13,2020-03-04,11,0.0,Wednesday,3,March,3,34.0,51,1,3.0,3.0,3912


<font color='blue'>MTL-AOS-5.1's Long and Lat put it in the middle of Central Park, and all 3 trips started and ended there.</font>

## 4. Removing all Station ID's with less than 100 hires
#### Most will be Bike Stations that were launched later in the year, so have not had the time to accrue hires like other stations.

In [ ]:
NYB2020_Stations.info()

In [49]:
NYB2020_Stations['uses_StartStation'] = NYB2020_Stations['uses_StartStation'].astype('int64')

In [50]:
#Creating a new column to display one of two flags, depending on the age of the person hiring the bike
NYB2020_Stations.loc[NYB2020_Stations['uses_StartStation'] > 100, 'Over_100'] = 'Y'
NYB2020_Stations.loc[NYB2020_Stations['uses_StartStation'] <= 100, 'Over_100'] = 'N'

In [53]:
NYB2020_Stations[['start_station_id', 'uses_StartStation', 'Over_100']].head(20)

,start_station_id,uses_StartStation,Over_100
0,504,59142,Y
1,3423,48873,Y
2,3687,73560,Y
3,346,38629,Y
4,372,8071,Y
5,394,28740,Y
6,3093,33556,Y
7,514,97415,Y
8,274,21312,Y
9,467,18195,Y


In [54]:
NYB2020_Stations['Over_100'].value_counts(dropna = False, ascending=True)

N        1000
Y    19505857
Name: Over_100, dtype: int64

In [55]:
NYB2020_Over100Uses = NYB2020_Stations[NYB2020_Stations['Over_100']=='Y']

## 5. Splitting out Unique Stations

In [58]:
#Creating a subset of the unique Start Stations
NYB2020_StationsUniqueStarts = NYB2020_Over100Uses.drop_duplicates(subset = ["start_station_id"])


In [59]:
#Showing that each station only shows up once
NYB2020_StationsUniqueStarts['start_station_id'].value_counts(dropna = False, ascending=True)

504     1
3404    1
3407    1
3105    1
278     1
       ..
3359    1
3100    1
3780    1
3782    1
4280    1
Name: start_station_id, Length: 1189, dtype: int64

In [20]:
NYB2020_StationsUniqueStarts.to_csv(os.path.join(path, '02 Data','Prepared Data', 'NYB2020_UsesByStation.csv'), index = False)

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,start_hour,start_date2,temp,rain_snow,day_of_week,day_of_week_number,month,month_number,TripMins,age,1col,uses_StartStation,Over_100,YearQTRs
19506837,148,2020-12-31 23:55:42.2030,2020-12-31 23:58:10.4400,464,E 56 St & 3 Ave,40.759345,-73.967597,456,E 53 St & Madison Ave,40.759711,-73.974023,36946,Subscriber,1994,1,23,2020-12-31,7,1.0,Thursday,4,December,12,2.0,26,1,24827,Y,4
19506838,76,2020-12-31 23:56:20.4840,2020-12-31 23:57:37.2340,312,Allen St & Stanton St,40.722055,-73.989111,265,Stanton St & Chrystie St,40.722293,-73.991475,40896,Subscriber,1994,2,23,2020-12-31,7,1.0,Thursday,4,December,12,1.0,26,1,47053,Y,4
19506839,1504,2020-12-31 23:56:28.5940,2021-01-01 00:21:33.2290,3628,Lenox Ave & W 117 St,40.802557,-73.949078,3137,5 Ave & E 72 St,40.772828,-73.966853,37804,Subscriber,1984,0,23,2020-12-31,7,1.0,Thursday,4,December,12,25.0,36,1,18901,Y,4
19506840,268,2020-12-31 23:56:35.5290,2021-01-01 00:01:03.8460,4099,E 169 St & Webster Ave,40.834015,-73.908863,4177,Prospect Ave & Jennings St,40.831762,-73.896853,47843,Customer,1993,2,23,2020-12-31,7,1.0,Thursday,4,December,12,4.0,27,1,1494,Y,4
19506841,636,2020-12-31 23:56:52.5540,2021-01-01 00:07:28.8400,3143,5 Ave & E 78 St,40.776321,-73.964274,3142,1 Ave & E 62 St,40.761227,-73.960940,40691,Subscriber,1976,1,23,2020-12-31,7,1.0,Thursday,4,December,12,11.0,44,1,38750,Y,4
19506842,1124,2020-12-31 23:57:15.6080,2021-01-01 00:15:59.8620,490,8 Ave & W 33 St,40.751551,-73.993934,490,8 Ave & W 33 St,40.751551,-73.993934,45654,Subscriber,1998,1,23,2020-12-31,7,1.0,Thursday,4,December,12,19.0,22,1,64669,Y,4
19506843,369,2020-12-31 23:57:17.1180,2021-01-01 00:03:27.1140,3288,E 88 St & 1 Ave,40.778301,-73.948813,3282,5 Ave & E 88 St,40.783070,-73.959390,49620,Customer,1969,0,23,2020-12-31,7,1.0,Thursday,4,December,12,6.0,51,1,32563,Y,4
19506844,1060,2020-12-31 23:57:23.8780,2021-01-01 00:15:04.4860,3977,Prospect Ave & Longwood Ave,40.818990,-73.902020,3977,Prospect Ave & Longwood Ave,40.818990,-73.902020,45758,Subscriber,1989,1,23,2020-12-31,7,1.0,Thursday,4,December,12,18.0,31,1,1320,Y,4
19506845,184,2020-12-31 23:57:33.1060,2021-01-01 00:00:37.6170,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,364,Lafayette Ave & Classon Ave,40.689004,-73.960239,37243,Subscriber,1994,1,23,2020-12-31,7,1.0,Thursday,4,December,12,3.0,26,1,8071,Y,4
19506846,1970,2020-12-31 23:57:47.3580,2021-01-01 00:30:38.0560,3169,Riverside Dr & W 82 St,40.787209,-73.981281,529,W 42 St & 8 Ave,40.757570,-73.990985,45168,Subscriber,2001,1,23,2020-12-31,7,1.0,Thursday,4,December,12,33.0,19,1,24405,Y,4


In [21]:
#Creating a subset of the unique End Stations
NYB2020_StationsUniqueEnds = NYB2020_Stations.drop_duplicates(subset = ["end_station_id"])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19505857 entries, 0 to 19506856
Data columns (total 29 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start_station_id         int16  
 4   start_station_name       object 
 5   start_station_latitude   float64
 6   start_station_longitude  float64
 7   end_station_id           int16  
 8   end_station_name         object 
 9   end_station_latitude     float64
 10  end_station_longitude    float64
 11  bikeid                   int32  
 12  usertype                 object 
 13  birth_year               int16  
 14  gender                   int8   
 15  start_hour               int8   
 16  start_date2              object 
 17  temp                     int8   
 18  rain_snow                float16
 19  day_of_week              object 
 20  day_of_week_number       int8   
 21  month 

In [22]:
NYB2020_StationsUniqueEnds.to_csv(os.path.join(path, '02 Data','Prepared Data', 'NYB2020_UniqueEnds.csv'), index = False)